In [1]:
# Importando bibliotecas
import os.path
import pandas as pd
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from google.auth.transport.requests import Request
from googleapiclient.discovery import build

**1. ATUALIZADO PLANILHA DO GOOGLE SHEETS**

In [16]:
# Carregando planilhas excel em dataframes
planilha_google_sheets_df = pd.read_excel('planilha_google_sheets.xlsx')
planilha_localizacao_df = pd.read_excel('planilha_localizacao.xlsx')

In [5]:
# Realiza o merge dos DataFrames com base na coluna "numero do processo"

planilha_atualizada_df = planilha_google_sheets_df.merge(planilha_localizacao_df,
    left_on='Nº DO PROCESSO',
    right_on='Nº DO PROCESSO',
    how='left'
)

In [7]:
# Tratamento da planilha após união dos dataframes
# Excluindo a coluna LOCALIZAÇÃO_x
planilha_atualizada_df = planilha_atualizada_df.drop('LOCALIZAÇÃO_x', axis=1) 
# Renomeando coluna com os dados atualizados para o nome inical da planilha no google sheets
planilha_atualizada_df.rename(columns={'LOCALIZAÇÃO_y': 'LOCALIZAÇÃO'}, inplace=True) 
# Ordenando s colunas conforme planilha inicial do google sheets
planilha_gooogle_sheets_atualizada_df = planilha_atualizada_df[
       ['OBJETO', 'ANO DA DEMANDA', 'Nº DO PROCESSO', 'DATA DO PROCESSO',
       'PROCESSO FINDOU EM', 'DIAS TRAMITADOS', 'MODALIDADE DE LICITAÇÃO',
       'TIPO', 'VALOR GLOBAL', 'SITUAÇÃO', 'LOCALIZAÇÃO', 'OBSERVAÇÃO']
]

**INÍCIO DO SCRIPT PARA ACESSAR O GOOGLE SHEETS**

In [9]:
# Escopo para o acesso - login
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
# Tentar ler as credenciais do arquivo token.json
creds = None
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)

# Se não houver credenciais válidas, solicite que o usuário o login
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)

    # Salvar as credenciais para a próxima execução
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

In [10]:
# Apontar o ID da planilha do Google Sheets
SPREADSHEET_ID = '1X3I2in2Cr6KmnAG4tMWzaucwz-nre84bXaSYXR-sH2g'
# Apontar o intervalo de células que deseja executar a leitura
RANGE_NAME = 'PLANILHA DESCRITIVA INTEGRAL!A:L'
# Ciação da variável 'serviço' para acessar a API do Google Sheets
service = build('sheets', 'v4', credentials=creds)
# Leitura dos dados da planilha do Google Sheets
result = service.spreadsheets().values().get(spreadsheetId=SPREADSHEET_ID, range=RANGE_NAME).execute()
values = result.get('values')

In [11]:
# Extrair os valores da coluna "LOCALIZAÇÃO" do DataFrame planilha_google_sheets-atualizada_df
dados_atualizados = planilha_gooogle_sheets_atualizada_df['LOCALIZAÇÃO'].tolist()
# Obter o intervalo das células da coluna "LOCALIZAÇÃO" para atualização
nome_intervalo = "PLANILHA DESCRITIVA INTEGRAL!K2:" + str(len(dados_atualizados) + 1)
# Formatar os dados para atualização no formato aceito pelo Google Sheets API
dados_formatados = [[localizacao] for localizacao in dados_atualizados]
#Agora você pode atualizar a coluna "LOCALIZAÇÃO" na planilha no Google Sheets
service.spreadsheets().values().update(
    spreadsheetId=SPREADSHEET_ID,
    range=nome_intervalo,
    valueInputOption='RAW',
    body={'values': dados_formatados}
).execute()

['NCIN', 'GABS', 'DSO', 'SCOM', 'SCOM', 'DAF', 'COLG', 'COFI', 'COFI', 'FISCON', 'CORI', 'COFI', 'SAENG', 'SCOM', 'CJUR', 'COFI', 'CJUR', 'NCIN', 'CORI', 'SCOM', 'NCIN', 'NCIN', 'COFI', 'DAF', 'DAF', 'SAENG', 'GABS', 'DAF', 'NCIN', 'CORV', 'SCOM', 'NCIN', 'DAF', 'ALMOX', 'CSEP', 'NCIN', 'COFI', 'NCIN', 'DAF', 'SAENG', 'COLG', 'COLG', 'DAF']
Coluna 'LOCALIZAÇÃO' atualizada com sucesso!


In [12]:
# Filtrar as linhas onde a coluna 'SITUAÇÃO' é diferente de 'FINALIZADO'

plan_atualizada_df = planilha_atualizada_df.loc[
    planilha_atualizada_df['SITUAÇÃO'] != 'FINALIZADO', ['OBJETO', 'Nº DO PROCESSO', 'LOCALIZAÇÃO']
]

In [14]:
# Salvar planilha em excel
plan_atualizada_df.to_excel('planilha_sheets_gerencial_daf.xlsx', index=False)  # O parâmetro index=False evita a inclusão do índice no arquivo

In [15]:
print('Coluna "LOCALIZAÇÃO",na planilha no google sheets, atualizada com sucesso!')

 Validadaação dos dados no google sheets realizada com sucesso!
